In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
def list_executions_from_Starnet_results(folder, case):
    filenames = glob.glob(pathname="*Input_Data*",root_dir=folder)
    executions = [fn.split("Input")[0][3:] + fn.split(case)[1][1:-4] for fn in filenames]
    return executions
def list_investments_candidates_from_execs(executions): 
    l = [execution.strip("PINTOOT")[1:] for execution in executions if (execution != "Network_Existing_Generation_Full" and execution != "Network_Full_Generation_Full" )]
    return np.unique(l)

In [3]:
case = "3-bus"
folder_read = f"../06.Benchmarking/E.Representative_days_based_on_Line_Benefits_OptModel/{case}/3.Out/DCOPF"

executions = list_executions_from_Starnet_results(folder_read,case)
ics = list_investments_candidates_from_execs(executions)

folder_write = f"{case}_DC_fy_LB"
per = 2030
sc = "sc01"

In [8]:
inp_full = pd.read_csv(f"{folder_read}/oT_Input_Data_{case}_Network_Existing_Generation_Full.csv")

#Obtain a set of desired variables to be kept
generation_types = pd.read_csv(f"../06.Benchmarking/E.Representative_days_based_on_Line_Benefits_OptModel/{case}/2.Par/oT_Data_Generation_{case}.csv")
techs_kept = ["Hydro", "Solar","Wind"]
variables = inp_full.Variable.unique()
variables_to_drop_i = np.unique(generation_types[~generation_types.Technology.isin(techs_kept)]["Unnamed: 0"])
variables_to_keep_i = [v for v in variables if v not in(variables_to_drop_i)]


 #Filter the main frame on desired parameters: 
f_i_datasets = ~inp_full.Dataset.str.startswith('Matrix')
f_i_tech = (inp_full.Variable.isin(variables_to_keep_i))
f_i = (f_i_datasets) & (f_i_tech)
#input_f = pd.DataFrame(all_input[f_i])
inp_f_p = inp_full[f_i].pivot(index = "LoadLevel",columns = ["Variable"],values = "Value")

for ic_col in ics:
    inp_f_p[ic_col] = 1
inp_f_p.to_csv(f"../Data/{folder_write}/input_f_{sc}_Full_Network_{per}.csv")



In [10]:
#Input stuff: other executions
for ic in ics:
    inp_ic = pd.read_csv(f"{folder_read}/oT_PINT_Input_Data_{case}_{ic}.csv")

    #Obtain a set of desired variables to be kept
    generation_types = pd.read_csv(f"../06.Benchmarking/E.Representative_days_based_on_Line_Benefits_OptModel/{case}/2.Par/oT_Data_Generation_{case}.csv")
    techs_kept = ["Hydro", "Solar","Wind"]
    variables = inp_ic.Variable.unique()
    variables_to_drop_i = np.unique(generation_types[~generation_types.Technology.isin(techs_kept)]["Unnamed: 0"])
    variables_to_keep_i = [v for v in variables if v not in(variables_to_drop_i)]


     #Filter the main frame on desired parameters: 
    f_i_datasets = ~inp_ic.Dataset.str.startswith('Matrix')
    f_i_tech = (inp_ic.Variable.isin(variables_to_keep_i))
    f_i = (f_i_datasets) & (f_i_tech)
    #input_f = pd.DataFrame(all_input[f_i])
    inp_f_p = inp_ic[f_i].pivot(index = "LoadLevel",columns = ["Variable"],values = "Value")

    for ic_col in ics:
        if ic_col == ic:
            inp_f_p[ic_col] = 1
        else:
            inp_f_p[ic_col] = 0
    inp_f_p.to_csv(f"../Data/{folder_write}/input_f_{sc}_{ic}_{per}.csv")


In [12]:
#Output stuff: Full network

#Read the files containing the data 
outp_existing = pd.read_csv(f"{folder_read}/oT_Output_Data_{case}_Network_Existing_Generation_Full.csv")
outp_full = pd.read_csv(f"{folder_read}/oT_Output_Data_{case}_Network_Full_Generation_Full.csv")

for key in ["SystemCosts"]: #,"PowerOutput","PowerFlow"]:
    #Filter to only keep the desired variables 

    f_o_1 = (outp_existing.Dataset == key)
    f_o_2 = (outp_full.Dataset == key)
    
    if key == "SystemCosts":
        variables_to_keep_o = ["vTotalCCost","vTotalECost","vTotalGCost","vTotalRCost"]

        f_o_1 =(outp_existing.Variable.isin(variables_to_keep_o))
        f_o_2 =(outp_full.Variable.isin(variables_to_keep_o))
    
    outp_existing_filtered = outp_existing[f_o_1]
    outp_full_filtered = outp_full[f_o_2]
    
    #Pivot
    outp_f_p_existing = outp_existing_filtered.pivot(index = "LoadLevel",columns=["Variable"],values = "Value")
    outp_f_p_full = outp_full_filtered.pivot(index = "LoadLevel",columns=["Variable"],values = "Value")
    
    #Calculate difference between existing and full network
    lbs = outp_f_p_existing - outp_f_p_full

    lbs.to_csv(f"../Data/{folder_write}/output_f_{sc}_Full_Network_{per}_{key}.csv")

In [13]:
#Output stuff: other executions 
for ic in ics:
    outp_PINT = pd.read_csv(f"{folder_read}/oT_PINT_Output_Data_{case}_{ic}.csv")
    outp_TOOT = pd.read_csv(f"{folder_read}/oT_TOOT_Output_Data_{case}_{ic}.csv")


    for key in ["SystemCosts"]: #,"PowerOutput","PowerFlow"]:
        #Filter to only keep the desired variables 

        f_o_1 = (outp_PINT.Dataset == key)
        f_o_2 = (outp_TOOT.Dataset == key)

        if key == "SystemCosts":
            variables_to_keep_o = ["vTotalCCost","vTotalECost","vTotalGCost","vTotalRCost"]

            f_o_1 =(outp_PINT.Variable.isin(variables_to_keep_o))
            f_o_2 =(outp_TOOT.Variable.isin(variables_to_keep_o))

        outp_PINT_filtered = outp_PINT[f_o_1]
        outp_TOOT_filtered = outp_TOOT[f_o_2]

        #Pivot
        outp_f_p_PINT= outp_PINT_filtered.pivot(index = "LoadLevel",columns=["Variable"],values = "Value")
        outp_f_p_TOOT = outp_TOOT_filtered.pivot(index = "LoadLevel",columns=["Variable"],values = "Value")

        #Calculate difference between existing and full network
        lbs = (outp_f_p_existing - outp_f_p_PINT)/2 - (outp_f_p_full - outp_f_p_TOOT)/2

        lbs.to_csv(f"../Data/{folder_write}/output_f_{sc}_{ic}_{per}_{key}.csv")

